# Build a variational autoencoder

Based on the ability to generate UK car registration numbers, one can build a dataset of training and test data. With a suitable vectorizer, one can go ahead and make an autoencoder.

The decoder part of the variational encoder can then be used to generate new reg numbers.

### References

https://blog.keras.io/building-autoencoders-in-keras.html

https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73

In [1]:
import sys
import math
import numpy as np
import keras
from keras import regularizers
from keras import layers
from keras import backend
from keras import optimizers
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegDvlaVectorizer

### Data generation

In [2]:
n_train = 10000
n_val = 1000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegDvlaVectorizer()

train_strs = [g.get_reg() for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
val_strs = [g.get_reg() for _ in range(n_val)]
val_strs = [x for x in val_strs if x not in set(train_strs)]  # sanity check
val_vecs = np.array([v.vectorize(x) for x in val_strs])
test_strs = [g.get_reg() for _ in range(n_test)]
test_strs = [x for x in test_strs if x not in set(train_strs) | set(val_strs)]  # sanity check
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print('input vector length = ' + str(vec_length))
print('actual number of validation samples = ' + str(len(val_strs)))
print('actual number of test samples = ' + str(len(test_strs)))

input vector length = 150
actual number of validation samples = 1000
actual number of test samples = 1000


### Variational autoencoder

Design the architecture

In [3]:
latent_dim = 3

### Define the encoder
inputs = keras.Input(shape=(vec_length,))
h = inputs
h = layers.Dense(vec_length, activation='relu')(h)
h = layers.Dense(50, activation='relu')(h)
h = layers.Dense(25, activation='relu')(h)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = backend.random_normal(shape=(backend.shape(z_mean)[0], latent_dim),
                                    mean=0., stddev=0.1)
    return z_mean + backend.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])

encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

### Define the decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = latent_inputs
x = layers.Dense(25, activation='relu')(x)
x = layers.Dense(50, activation='relu')(x)
x = layers.Dense(vec_length, activation='relu')(x)
x = layers.Dense(vec_length, activation='sigmoid')(x)
outputs = x
decoder = keras.Model(latent_inputs, outputs, name='decoder')

outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

### Define the loss function
reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= vec_length
kl_loss = 1 + z_log_sigma - backend.square(z_mean) - backend.exp(z_log_sigma)
kl_loss = backend.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = backend.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

Do some training. The training below has a hard time optimizing when the latent dimension is less than 15, but for some of the techniques used later in the notebook, a much smaller latent dimension is required.

In [4]:
opt = optimizers.Adamax(learning_rate=0.004)  # larger learning rate for faster descent
vae.compile(optimizer=opt, loss='binary_crossentropy')
vae.fit(train_vecs, train_vecs,
        epochs=200,
        batch_size=8,
        shuffle=True,
        validation_data=(val_vecs, val_vecs))

opt = optimizers.Adamax(learning_rate=0.001)  # slower learning rate for approaching local min
vae.compile(optimizer=opt, loss='binary_crossentropy')
vae.fit(train_vecs, train_vecs,
        epochs=200,
        batch_size=8,
        shuffle=True,
        validation_data=(val_vecs, val_vecs))

Epoch 1/200
1250/1250 [==============================] - 1s 1ms/step - loss: 28.4095 - val_loss: 26.8174
Epoch 2/200
1250/1250 [==============================] - 1s 902us/step - loss: 26.4031 - val_loss: 25.6767
Epoch 3/200
1250/1250 [==============================] - 1s 895us/step - loss: 25.0307 - val_loss: 24.5068
Epoch 4/200
1250/1250 [==============================] - 1s 888us/step - loss: 23.7883 - val_loss: 23.4015
Epoch 5/200
1250/1250 [==============================] - 1s 883us/step - loss: 22.9773 - val_loss: 22.8238
Epoch 6/200
1250/1250 [==============================] - 1s 890us/step - loss: 22.5699 - val_loss: 22.5616
Epoch 7/200
1250/1250 [==============================] - 1s 889us/step - loss: 22.3223 - val_loss: 22.3668
Epoch 8/200
1250/1250 [==============================] - 1s 904us/step - loss: 22.1445 - val_loss: 22.3599
Epoch 9/200
1250/1250 [==============================] - 1s 885us/step - loss: 22.0490 - val_loss: 22.1736
Epoch 10/200
1250/1250 [===============

1250/1250 [==============================] - 1s 917us/step - loss: 20.3480 - val_loss: 21.4401
Epoch 78/200
1250/1250 [==============================] - 1s 858us/step - loss: 20.3267 - val_loss: 21.4753
Epoch 79/200
1250/1250 [==============================] - 1s 850us/step - loss: 20.3169 - val_loss: 21.4881
Epoch 80/200
1250/1250 [==============================] - 1s 872us/step - loss: 20.3148 - val_loss: 21.4838
Epoch 81/200
1250/1250 [==============================] - 1s 877us/step - loss: 20.2850 - val_loss: 21.5164
Epoch 82/200
1250/1250 [==============================] - 1s 868us/step - loss: 20.2914 - val_loss: 21.4499
Epoch 83/200
1250/1250 [==============================] - 1s 871us/step - loss: 20.2703 - val_loss: 21.4730
Epoch 84/200
1250/1250 [==============================] - 1s 875us/step - loss: 20.2760 - val_loss: 21.4375
Epoch 85/200
1250/1250 [==============================] - 1s 868us/step - loss: 20.2755 - val_loss: 21.4693
Epoch 86/200
1250/1250 [=================

1250/1250 [==============================] - 1s 890us/step - loss: 19.7981 - val_loss: 21.7917
Epoch 153/200
1250/1250 [==============================] - 1s 880us/step - loss: 19.7724 - val_loss: 21.8600
Epoch 154/200
1250/1250 [==============================] - 1s 883us/step - loss: 19.8014 - val_loss: 21.8895
Epoch 155/200
1250/1250 [==============================] - 1s 881us/step - loss: 19.7807 - val_loss: 21.8136
Epoch 156/200
1250/1250 [==============================] - 1s 939us/step - loss: 19.7842 - val_loss: 21.9327
Epoch 157/200
1250/1250 [==============================] - 1s 919us/step - loss: 19.7505 - val_loss: 21.8528
Epoch 158/200
1250/1250 [==============================] - 1s 924us/step - loss: 19.7680 - val_loss: 21.7932
Epoch 159/200
1250/1250 [==============================] - 1s 935us/step - loss: 19.7549 - val_loss: 21.8667
Epoch 160/200
1250/1250 [==============================] - 1s 918us/step - loss: 19.7722 - val_loss: 21.8698
Epoch 161/200
1250/1250 [========

Epoch 28/200
1250/1250 [==============================] - 1s 940us/step - loss: 19.1856 - val_loss: 22.0427
Epoch 29/200
1250/1250 [==============================] - 1s 913us/step - loss: 19.1804 - val_loss: 22.1042
Epoch 30/200
1250/1250 [==============================] - 1s 956us/step - loss: 19.1780 - val_loss: 22.0471
Epoch 31/200
1250/1250 [==============================] - 1s 959us/step - loss: 19.1771 - val_loss: 22.0569
Epoch 32/200
1250/1250 [==============================] - 1s 878us/step - loss: 19.1771 - val_loss: 22.0679
Epoch 33/200
1250/1250 [==============================] - 1s 935us/step - loss: 19.1723 - val_loss: 22.0645
Epoch 34/200
1250/1250 [==============================] - 1s 956us/step - loss: 19.1658 - val_loss: 22.1152
Epoch 35/200
1250/1250 [==============================] - 1s 955us/step - loss: 19.1848 - val_loss: 22.0783
Epoch 36/200
1250/1250 [==============================] - 1s 976us/step - loss: 19.1658 - val_loss: 22.0265
Epoch 37/200
1250/1250 [====

Epoch 104/200
1250/1250 [==============================] - 1s 899us/step - loss: 19.0390 - val_loss: 22.3502
Epoch 105/200
1250/1250 [==============================] - 1s 887us/step - loss: 19.0163 - val_loss: 22.3072
Epoch 106/200
1250/1250 [==============================] - 1s 891us/step - loss: 19.0319 - val_loss: 22.3135
Epoch 107/200
1250/1250 [==============================] - 1s 915us/step - loss: 19.0335 - val_loss: 22.3241
Epoch 108/200
1250/1250 [==============================] - 1s 846us/step - loss: 19.0186 - val_loss: 22.3340
Epoch 109/200
1250/1250 [==============================] - 1s 861us/step - loss: 19.0222 - val_loss: 22.3382
Epoch 110/200
1250/1250 [==============================] - 1s 837us/step - loss: 19.0271 - val_loss: 22.3764
Epoch 111/200
1250/1250 [==============================] - 1s 838us/step - loss: 19.0226 - val_loss: 22.3417
Epoch 112/200
1250/1250 [==============================] - 1s 837us/step - loss: 19.0258 - val_loss: 22.3523
Epoch 113/200
1250/

1250/1250 [==============================] - 1s 891us/step - loss: 18.9201 - val_loss: 22.4809
Epoch 180/200
1250/1250 [==============================] - 1s 924us/step - loss: 18.9233 - val_loss: 22.5649
Epoch 181/200
1250/1250 [==============================] - 1s 878us/step - loss: 18.9164 - val_loss: 22.5370
Epoch 182/200
1250/1250 [==============================] - 1s 880us/step - loss: 18.9214 - val_loss: 22.5473
Epoch 183/200
1250/1250 [==============================] - 1s 867us/step - loss: 18.9177 - val_loss: 22.5714
Epoch 184/200
1250/1250 [==============================] - 1s 928us/step - loss: 18.9003 - val_loss: 22.5377
Epoch 185/200
1250/1250 [==============================] - 1s 930us/step - loss: 18.9147 - val_loss: 22.5802
Epoch 186/200
1250/1250 [==============================] - 1s 873us/step - loss: 18.9002 - val_loss: 22.5343
Epoch 187/200
1250/1250 [==============================] - 1s 861us/step - loss: 18.9151 - val_loss: 22.5262
Epoch 188/200
1250/1250 [========

Evaluation

In [5]:
decoded_regs = vae.predict(test_vecs)
print(test_strs[:10])
recovered = [v.recover(x) for x in decoded_regs]
print(recovered[:10])
acc = np.sum([x == y for x, y in zip(recovered, test_strs)]) / len(test_strs)
print('overall accuracy = ' + str(acc))
print('by digit accuracy:')
for i_digit in range(len(test_strs[0])):
    acc = np.sum([x[i_digit] == y[i_digit] for x, y in zip(recovered, test_strs)]) / len(test_strs)
    print('    ' + str(acc))

['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB17WIY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
['OJ33GNY', 'WP83GRK', 'OU19PVV', 'GX41GTW', 'KP91ZGB', 'OX47DKA', 'RP17PJZ', 'WG89SVH', 'SP57VGK', 'MR77MUW']
overall accuracy = 0.0
by digit accuracy:
    0.095
    0.162
    0.988
    0.989
    0.086
    0.073
    0.101


### Use decoder as generator

By directly sampling in the latent space, and running through the decoder, one effectively has a data generator.

In [6]:
[v.recover(x) for x in decoder.predict(np.random.random((5, latent_dim)))]

['MH38JWD', 'HE04JWL', 'MJ95OVA', 'KJ04ERK', 'VV34ILR']

One can do some sampling and look at some stats to see what kind of data comes out. For instance, the number of reg numbers with pair of integers below 50 and above 50

In [32]:
random_vectors = np.random.random((1000, latent_dim)) - 0.5
generated_regs = [v.recover(x) for x in decoder.predict(random_vectors)]
n_reg_below_50 = len([x for x in generated_regs if int(x[2:4]) < 50])
n_reg_above_50 = len([x for x in generated_regs if int(x[2:4]) >= 50])
print('reg numbers below 50 = ' + str(n_reg_below_50))
print('reg numbers above 50 = ' + str(n_reg_above_50))

reg numbers below 50 = 412
reg numbers above 50 = 588


### Avoid generating any training data

There is a chance that a random generation of data might produce one of the actual registration numbers used in training. However, any input registrations are represented as a probability distribution in the latent space; so one could require that any sample vector used in the latent space to be 'sufficiently far' down the tails of all the recorded distributions.

The distributions of the training data in the latent space are readily available:

In [8]:
# Get the distributions for each reg number in the data
all_strs = train_strs + val_strs + test_strs
all_vecs = np.array([v.vectorize(x) for x in all_strs])
means, log_sigmas, _ = encoder.predict(all_vecs)
distn_dict = {x: (y,z) for x, y, z in zip(all_strs, means, np.exp(log_sigmas))}  # dict<reg no: (mean, sigma)>
example_reg = list(distn_dict.keys())[0]
print('distribution for input data registration ' + str(example_reg))
print('    mu = ' + str(distn_dict[example_reg][0]))
print('    diagonal sigmas = ' + str(distn_dict[example_reg][1]))
# for a multivariate distribution ~ N(mean, Var) valued in the latent space, 
# mean is the first vector below, and Var is a diagonal matrix, the square root
# of the diagonal entries being given in the second vector.
# If the KL loss function pulls the distributions to be close to N(0,1)

distribution for input data registration YK66BIQ
    mu = [-0.40926898 -0.9612769   0.33374935]
    diagonal sigmas = [0.32369483 0.3985572  0.27955106]


One can work out a rough area in the latent space that the data vectors occupy. This could be important; certainly any point in the latent space can be used (since there is a distribution over the space), but how valid are samples from far outside the region covered by the 'heavy' parts of the obtained distributions?

In [22]:
all_means_mtr = np.array([x[0] for x in distn_dict.values()])
print('minimum of means over each latent dimension:')
print(np.min(all_means_mtr, axis=0))
print('maximum of means over each latent dimension:')
print(np.max(all_means_mtr, axis=0))

minimum of means over each latent dimension:
[-1.831372  -1.5763952 -1.958655 ]
maximum of means over each latent dimension:
[1.9591135 1.7558839 2.0847764]


### Generating subsets of data with certain characteristics

As a more general example of 'avoid generating any training data', consider the case of 'avoid generating data with certain characteristics'. Since the latent space has a joint probability density function over it, this can be answered in a statistical manner. Consider the distributions of all data, that represent numbers below 50. Recall all 7 digits are alpha numeric, with only the third and fourth digits being numeric.

In [9]:
distns_below_50 = {k: v for k, v in distn_dict.items() if int(k[2:4]) < 50}
print('number of distributions with number below 50 = ' + str(len(distns_below_50)))

number of distributions with number below 50 = 6103


and use these to find only random vectors that are 'sufficiently far away' from the means of those reg numbers with numbers below 50. So the 'characteristic' chosen, will be for the number to be above 50; the reg numbers are of the form AAXXAAA, so the characteristic is that XX is greater than 50.

Employing a cheap-and-cheerful example of rejection sampling, and a cheap-and-cheerful example of statistical hypothesis testing, one can obtain the a set of random vectors that are 'sufficiently far' in the tails of the distributions belonging to those data vectors whose registration numbers are below 50.

In [59]:
# magnitude tuned so that it is roughly the volume covered by the means of the distributions
random_vectors = 5 * (np.random.random((1000, latent_dim)) - 0.5)
generated_regs = [v.recover(x) for x in decoder.predict(random_vectors)]
print('reg numbers below 50 = ' + str(len([x for x in generated_regs if int(x[2:4]) < 50])))
print('reg numbers above 50 = ' + str(len([x for x in generated_regs if int(x[2:4]) >= 50])))

reg numbers below 50 = 530
reg numbers above 50 = 470


In [60]:
# for each of the random vectors, find the smallest Mahalanobis distance from all the distributions with
# registration numeric part less than 50
means_mtr = np.array([mu for mu, sigma in distns_below_50.values()])
sigma_mtr = np.array([sigma for mu, sigma in distns_below_50.values()])
min_mahalanobis_dists = [np.min(np.linalg.norm((x - means_mtr) / sigma_mtr, axis=1))
                         for x in random_vectors]

# set a threshold on the Mahalanobis distance (which roughly corresponds to a hypothesis
# test significance level) to perform some crude rejection sampling
mahalanobis_threshold = 1.0
# I'm not convinced the latent_dim should be in the denominator here....
filtered_random_vectors = [x for x, y in zip(random_vectors, np.array(min_mahalanobis_dists)/latent_dim) 
                           if y > mahalanobis_threshold]
print('theoretical probability of sample from normal distribution having mahalanobis distance greater')
print('than mahalanobis threshold = ' +str(2*(1 - math.erf(0.5 * mahalanobis_threshold))))
print('number of filtered random vectors = ' + str(len(filtered_random_vectors)))

theoretical probability of sample from normal distribution having mahalanobis distance greater
than mahalanobis threshold = 0.959000244373907
number of filtered random vectors = 228


In [61]:
if len(filtered_random_vectors) > 0:
    filtered_generated_regs = [v.recover(x) for x in decoder.predict(np.array(filtered_random_vectors))]
    filtered_n_reg_above_50 = len([x for x in filtered_generated_regs if int(x[2:4]) > 50])
    print('number of filtered reg numbers with numeric part above 50 = ' + str(filtered_n_reg_above_50))
    print('proportion of filtered reg numbers obeying chosen characteristic = '
          + str(filtered_n_reg_above_50/len(filtered_random_vectors)))
else:
    print('no filtered random vectors to work with')

number of filtered reg numbers with numeric part above 50 = 215
proportion of filtered reg numbers obeying chosen characteristic = 0.9429824561403509
